In [1]:
pip install pgmpy


Defaulting to user installation because normal site-packages is not writeable
  Using cached pgmpy-0.1.25-py3-none-any.whl.metadata (6.4 kB)
  Using cached networkx-3.3-py3-none-any.whl.metadata (5.1 kB)
  Using cached numpy-2.0.0-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached scipy-1.14.0-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached scikit_learn-1.5.0-cp311-cp311-win_amd64.whl.metadata (11 kB)
  Using cached pandas-2.2.2-cp311-cp311-win_amd64.whl.metadata (19 kB)
  Using cached pyparsing-3.1.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached torch-2.3.1-cp311-cp311-win_amd64.whl.metadata (26 kB)
  Using cached statsmodels-0.14.2-cp311-cp311-win_amd64.whl.metadata (9.5 kB)
  Using cached tqdm-4.66.4-py3-none-any.whl.metadata (57 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD

# Define the structure of the Bayesian Network
model = BayesianNetwork([('Burglary', 'Alarm'),
                         ('Earthquake', 'Alarm'),
                         ('Alarm', 'DavidCalls'),
                         ('Alarm', 'SophiaCalls')])

# Define the Conditional Probability Distributions (CPDs)
cpd_burglary = TabularCPD(variable='Burglary', variable_card=2, values=[[0.99], [0.01]])
cpd_earthquake = TabularCPD(variable='Earthquake', variable_card=2, values=[[0.98], [0.02]])

# CPD for Alarm given Burglary and Earthquake
cpd_alarm = TabularCPD(variable='Alarm', variable_card=2,
                       values=[[0.999, 0.71, 0.06, 0.05],
                               [0.001, 0.29, 0.94, 0.95]],
                       evidence=['Burglary', 'Earthquake'],
                       evidence_card=[2, 2])

# CPD for DavidCalls given Alarm
cpd_david_calls = TabularCPD(variable='DavidCalls', variable_card=2,
                             values=[[0.95, 0.1],
                                     [0.05, 0.9]],
                             evidence=['Alarm'],
                             evidence_card=[2])

# CPD for SophiaCalls given Alarm
cpd_sophia_calls = TabularCPD(variable='SophiaCalls', variable_card=2,
                              values=[[0.99, 0.3],
                                      [0.01, 0.7]],
                              evidence=['Alarm'],
                              evidence_card=[2])

# Associating the CPDs with the network structure
model.add_cpds(cpd_burglary, cpd_earthquake, cpd_alarm, cpd_david_calls, cpd_sophia_calls)

# Check if the model is correctly structured and consistent
assert model.check_model()

# Print the network structure
print("Bayesian Network structure:")
for edge in model.edges():
    print(edge)

# Optionally, you can also print the CPDs
print("\nCPDs:")
for cpd in model.get_cpds():
    print(cpd)

# You can perform other operations with the Bayesian Network model here...


C:\Users\CH.EN.U4AIE22059\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Bayesian Network structure:
('Burglary', 'Alarm')
('Alarm', 'DavidCalls')
('Alarm', 'SophiaCalls')
('Earthquake', 'Alarm')

CPDs:
+-------------+------+
| Burglary(0) | 0.99 |
+-------------+------+
| Burglary(1) | 0.01 |
+-------------+------+
+---------------+------+
| Earthquake(0) | 0.98 |
+---------------+------+
| Earthquake(1) | 0.02 |
+---------------+------+
+------------+---------------+---------------+---------------+---------------+
| Burglary   | Burglary(0)   | Burglary(0)   | Burglary(1)   | Burglary(1)   |
+------------+---------------+---------------+---------------+---------------+
| Earthquake | Earthquake(0) | Earthquake(1) | Earthquake(0) | Earthquake(1) |
+------------+---------------+---------------+---------------+---------------+
| Alarm(0)   | 0.999         | 0.71          | 0.06          | 0.05          |
+------------+---------------+---------------+---------------+---------------+
| Alarm(1)   | 0.001         | 0.29          | 0.94          | 0.95          

In [3]:
from itertools import product

# Define the variables and their states
variables = ['Burglary', 'Earthquake', 'Alarm', 'DavidCalls', 'SophiaCalls']
variable_states = {
    'Burglary': [0, 1],     # 0 = false, 1 = true
    'Earthquake': [0, 1],
    'Alarm': [0, 1],
    'DavidCalls': [0, 1],
    'SophiaCalls': [0, 1]
}

# Define CPDs as given in your example
cpds = {
    'Burglary': [0.99, 0.01],
    'Earthquake': [0.98, 0.02],
    'Alarm': {
        (0, 0): [0.999, 0.001],
        (0, 1): [0.71, 0.29],
        (1, 0): [0.06, 0.94],
        (1, 1): [0.05, 0.95]
    },
    'DavidCalls': {
        0: [0.95, 0.05],
        1: [0.1, 0.9]
    },
    'SophiaCalls': {
        0: [0.99, 0.01],
        1: [0.3, 0.7]
    }
}

def joint_distribution(user_input):
    # Initialize joint distribution as a dictionary
    joint_distribution = {}

    # Iterate over all possible combinations of variable states
    for states in product(*[variable_states[var] for var in variables]):
        states_dict = dict(zip(variables, states))
        
        # Check if the states match the user input
        match = True
        for var, input_state in zip(variables, user_input):
            if states_dict[var] != input_state:
                match = False
                break
        
        if match:
            
            joint_prob = cpds['Burglary'][states_dict['Burglary']] \
                        * cpds['Earthquake'][states_dict['Earthquake']] \
                        * cpds['Alarm'][(states_dict['Burglary'], states_dict['Earthquake'])][states_dict['Alarm']] \
                        * cpds['DavidCalls'][states_dict['Alarm']][states_dict['DavidCalls']] \
                        * cpds['SophiaCalls'][states_dict['Alarm']][states_dict['SophiaCalls']]
            
            
            state_key = tuple(f"{var}={states_dict[var]}" for var in variables)
            
            
            joint_distribution[state_key] = joint_prob
    
    return joint_distribution

# user input

user_input = [0, 0, 1, 1, 1]  
result = joint_distribution(user_input)

# Printing the output
print("Joint Distribution:")
for key, value in result.items():
    print(f"P({', '.join(key)}) = {value}")


Joint Distribution:
P(Burglary=0, Earthquake=0, Alarm=1, DavidCalls=1, SophiaCalls=1) = 0.000611226


### Problem - 3 

### Maximum likelihood Estimation

In [11]:
# Import necessary libraries
import numpy as np
import pandas as pd
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator

# Define the structure of the Bayesian Network
model = BayesianNetwork([
    ('Burglary', 'Alarm'),
    ('Earthquake', 'Alarm'),
    ('Alarm', 'DavidCalls'),
    ('Alarm', 'SophiaCalls')
])

# Create a sample dataset
# This data should be replaced with actual observations
data = pd.DataFrame(data={
    'Burglary': np.random.randint(2, size=1000),
    'Earthquake': np.random.randint(2, size=1000),
    'Alarm': np.random.randint(2, size=1000),
    'DavidCalls': np.random.randint(2, size=1000),
    'SophiaCalls': np.random.randint(2, size=1000)
})

# Apply Maximum Likelihood Estimation to learn parameters
model.fit(data, estimator=MaximumLikelihoodEstimator)

# Print the learned CPDs (Conditional Probability Distributions)
for cpd in model.get_cpds():
    print(f"CPD of {cpd.variable}:")
    print(cpd)


CPD of Burglary:
+-------------+-------+
| Burglary(0) | 0.532 |
+-------------+-------+
| Burglary(1) | 0.468 |
+-------------+-------+
CPD of Alarm:
+------------+---------------------+-----+---------------------+
| Burglary   | Burglary(0)         | ... | Burglary(1)         |
+------------+---------------------+-----+---------------------+
| Earthquake | Earthquake(0)       | ... | Earthquake(1)       |
+------------+---------------------+-----+---------------------+
| Alarm(0)   | 0.47038327526132406 | ... | 0.48770491803278687 |
+------------+---------------------+-----+---------------------+
| Alarm(1)   | 0.5296167247386759  | ... | 0.5122950819672131  |
+------------+---------------------+-----+---------------------+
CPD of Earthquake:
+---------------+-------+
| Earthquake(0) | 0.511 |
+---------------+-------+
| Earthquake(1) | 0.489 |
+---------------+-------+
CPD of DavidCalls:
+---------------+--------------------+--------------------+
| Alarm         | Alarm(0)          

# mahe_code

In [9]:
import numpy as np
import pandas as pd
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.inference import VariableElimination


model = BayesianNetwork([
    ('B', 'A'),  # Burglary causes Alarm
    ('E', 'A'),  # Earthquake causes Alarm
    ('A', 'D'),  # Alarm causes David to call
    ('A', 'S')   # Alarm causes Sophia to call
])

data = pd.DataFrame(np.random.randint(0, 2, size=(1000, 5)), columns=['B', 'E', 'A', 'D', 'S'])

model.fit(data, estimator=MaximumLikelihoodEstimator)

for cpd in model.get_cpds():
    print(f"\nCPD of {cpd.variable}:")
    print(cpd)

inference = VariableElimination(model)

result = inference.query(variables=['B'], evidence={'D': 1, 'S': 1})

print("\nP(B | D=true, S=true):")

print(result)


CPD of B:
+------+-------+
| B(0) | 0.512 |
+------+-------+
| B(1) | 0.488 |
+------+-------+

CPD of A:
+------+---------------------+-----+--------------------+
| B    | B(0)                | ... | B(1)               |
+------+---------------------+-----+--------------------+
| E    | E(0)                | ... | E(1)               |
+------+---------------------+-----+--------------------+
| A(0) | 0.5294117647058824  | ... | 0.5194805194805194 |
+------+---------------------+-----+--------------------+
| A(1) | 0.47058823529411764 | ... | 0.4805194805194805 |
+------+---------------------+-----+--------------------+

CPD of E:
+------+-------+
| E(0) | 0.529 |
+------+-------+
| E(1) | 0.471 |
+------+-------+

CPD of D:
+------+---------------------+--------------------+
| A    | A(0)                | A(1)               |
+------+---------------------+--------------------+
| D(0) | 0.48906560636182905 | 0.5050301810865191 |
+------+---------------------+--------------------+
| D(